# importing

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import datetime

from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.seasonal import seasonal_decompose
sns.set(style="darkgrid")

In [ ]:
global_temp_df = pd.read_csv('data/global_temp_df.csv')

In [ ]:
co2_concentration_df = pd.read_csv('data/co2_concentration_df.csv')